In [2]:
#%pip install searchtweets pandas

In [ ]:
from datetime import datetime
from searchtweets import gen_request_parameters, ResultStream
import pandas as pd

In [2]:
# your twitter bearer token
token = "YOUR-TWITTER-TOKEN"

In [ ]:
# hastags we want to search for
hashtags = ["#veganfood", "veganideas", "#veganrecipes", "veganuary", "veganuary2022"]

In [4]:
# define the end time of your search window, twitter allows you to look for one week of tweets for free
end_time = datetime(2022, 1, 18) 

In [165]:
# define our search query, and the information we want to get back
search_query = gen_request_parameters(
    query=f"lang:en ({' OR '.join(hashtags)}) -is:retweet -is:reply", 
    granularity=None,
    end_time=end_time.strftime("%Y%m%d%H%M"),
    tweet_fields=",".join(
        [
            "author_id",
            "conversation_id",
            "created_at",
            "id",
            "public_metrics",
            "text",
            "geo",
            "lang",
            "possibly_sensitive",
            "referenced_tweets",
            "source",
            # "organic_metrics",
            "context_annotations",
            "entities",
            "withheld",
        ]
    ),
    user_fields=",".join(
        ["id", "name", "public_metrics", "username", "verified", "description"]
    ),
    place_fields=",".join(["geo", "name", "full_name", "place_type"]),
    results_per_call=100,
    expansions="author_id",
)

In [166]:
# get tweets
rs = ResultStream(
    endpoint="https://api.twitter.com/2/tweets/search/recent",
    request_parameters=search_query,
    bearer_token=token,
    max_tweets=10000,
)

pages = list(rs.stream())

In [172]:
# add user information to tweet
tweets = []
for page in pages[:]:
    for tweet in page["data"]:
        user_dict = None
        for user in page["includes"]["users"]:
            if user["id"] == tweet["author_id"]:
                user_dict = user
                break
        if user_dict is None:
            print(tweet)
            raise Exception
        
        tweets.append({**tweet, "user": user_dict})

In [173]:
# how many tweets did we retrieve
len(tweets)

7899

In [199]:
# load tweets into a pandas DataFrame
tweets_df = pd.DataFrame(tweets)

In [200]:
# save tweets as a json file
tweets_df.to_json("tweets.json")

In [4]:
# have a look at the data
pd.read_json("tweets.json")

,entities,context_annotations,conversation_id,lang,text,created_at,possibly_sensitive,author_id,source,public_metrics,id,user,referenced_tweets,geo
0,"{'hashtags': [{'start': 134, 'end': 145, 'tag'...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",1483227498603233280,en,Scottish fish farms supplying salmon to U.K su...,2022-01-17 23:59:27+00:00,False,800414768024518656,Tweepsmap,"{'retweet_count': 5, 'reply_count': 2, 'like_c...",1483227498603233280,{'description': 'Injustice anywhere is a threa...,None,None
1,"{'hashtags': [{'start': 178, 'end': 188, 'tag'...","[{'domain': {'id': '65', 'name': 'Interests an...",1483225300662198272,en,My all time favourite! We call it Chilli Bang ...,2022-01-17 23:50:43+00:00,False,276979553,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1483225300662198272,"{'description': 'Tap dancing vegan 💚', 'verifi...",None,None
2,"{'hashtags': [{'start': 103, 'end': 109, 'tag'...","[{'domain': {'id': '65', 'name': 'Interests an...",1483224161904431104,en,"My favorite salad. Roasted delicata squash, fa...",2022-01-17 23:46:11+00:00,False,16227985,Instagram,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1483224161904431104,"{'description': 'Plant/cat/dog mom, gardening,...",None,None
3,"{'hashtags': [{'start': 122, 'end': 128, 'tag'...","[{'domain': {'id': '65', 'name': 'Interests an...",1483224030589108224,en,These vegan snacks were…interesting 🤯 Vegan Je...,2022-01-17 23:45:40+00:00,False,1427416003533758464,Twitter for iPhone,"{'retweet_count': 1, 'reply_count': 1, 'like_c...",1483224030589108224,{'description': '🎙A Podcast about everything a...,None,None
4,"{'hashtags': [{'start': 247, 'end': 257, 'tag'...","[{'domain': {'id': '65', 'name': 'Interests an...",1483223929036673024,en,"We know what's good, but we love what tastes g...",2022-01-17 23:45:16+00:00,False,1479155089273241600,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1483223929036673024,"{'description': '', 'verified': False, 'public...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7894,"{'hashtags': [{'start': 66, 'end': 76, 'tag': ...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",1480927423596224512,en,I pledged to not buy any of the new vegan food...,2022-01-11 15:39:46+00:00,False,3371265334,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1480927423596224512,"{'username': 'DeanColeWriter', 'description': ...",None,None
7895,"{'hashtags': [{'start': 20, 'end': 34, 'tag': ...","[{'domain': {'id': '65', 'name': 'Interests an...",1480927076794478592,en,It’s Taco Tuesday!! #Veganuary2022 #tacotuesda...,2022-01-11 15:38:24+00:00,False,1407406921213218816,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1480927076794478592,"{'username': 'justjenrenee', 'description': 'H...",None,None
7896,"{'hashtags': [{'start': 143, 'end': 148, 'tag'...",None,1480926881646055424,en,Hate is a close cousin to indifference when it...,2022-01-11 15:37:37+00:00,False,1247542635352469504,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1480926881646055424,"{'username': 'WendyRaymond123', 'description':...",None,None
7897,"{'urls': [{'start': 51, 'end': 74, 'url': 'htt...","[{'domain': {'id': '65', 'name': 'Interests an...",1480926878533832704,en,How To Stay Vegan Beyond Veganuary | THE FARM ...,2022-01-11 15:37:36+00:00,False,22035674,Twitter for iPhone,"{'retweet_count': 7, 'reply_count': 0, 'like_c...",1480926878533832704,"{'username': 'FARMUSA', 'description': 'Farm A...",None,None
